# Visualising French regional elections (2021), first round

Data (export 2018): https://www.data.gouv.fr/fr/datasets/contours-des-regions-francaises-sur-openstreetmap/

https://www.data.gouv.fr/en/datasets/elections-regionales-2021-resultats-du-1er-tour-1/

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
import json
from datetime import datetime
import plotly.express as px

%matplotlib notebook


### Data loading

In [2]:
df_map = gpd.read_file('regions-20180101-shp/regions-20180101.shp')
df_data = pd.read_excel('resultats-par-niveau-reg-t1-france-entiere-2021-06-22-11h19.xlsx')

display(df_map.head())
display(df_data.head())

,code_insee,nom,nuts2,wikipedia,surf_km2,geometry
0,04,La Réunion,FR94,fr:La Réunion,2505.0,"MULTIPOLYGON (((55.21643 -21.03904, 55.21652 -..."
1,94,Corse,FR83,fr:Corse,8722.0,"MULTIPOLYGON (((8.53996 42.23689, 8.54030 42.2..."
2,02,Martinique,FR92,fr:Martinique,1089.0,"MULTIPOLYGON (((-61.22908 14.82247, -61.22895 ..."
3,11,Île-de-France,FR10,fr:Île-de-France,12069.0,"POLYGON ((1.44624 49.04639, 1.44945 49.04765, ..."
4,32,Hauts-de-France,FR22,fr:Nord-Pas-de-Calais-Picardie,31935.0,"MULTIPOLYGON (((1.37983 50.06518, 1.38000 50.0..."


,Code de la région,Libellé de la région,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,...,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141
0,44,Grand Est,3829952,2695773,70.39,1134179,29.61,34389,0.90,3.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,75,Nouvelle-Aquitaine,4352306,2787124,64.04,1565182,35.96,39346,0.90,2.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84,Auvergne-Rhône-Alpes,5403344,3642126,67.41,1761218,32.59,30859,0.57,1.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27,Bourgogne-Franche-Comté,1967032,1281027,65.12,686005,34.88,21293,1.08,3.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,53,Bretagne,2493602,1601187,64.21,892415,35.79,24211,0.97,2.71,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data cleaning and combine the two dataframes

Votants = Blancs + Nuls + Exprimés

In [3]:
df_map = df_map.drop(columns=['code_insee', 'nuts2', 'wikipedia', 'surf_km2'])
df_map = df_map[df_map['nom'] != 'Mayotte']

# The first 16 columns are the general features, then it goes by blocks of 9 columns
nb_columns = df_data.shape[1]
for i in range(0, int((nb_columns-16)/9)-1):
    start = 25 + i * 9
    df_data.rename(columns = {df_data.columns[start]:f'N°Liste_{i}'}, inplace = True)
    df_data.rename(columns = {df_data.columns[start+1]:f'Nuance Liste_{i}'}, inplace = True)
    df_data.rename(columns = {df_data.columns[start+2]:f'Libellé Abrégé Liste_{i}'}, inplace = True)
    df_data.rename(columns = {df_data.columns[start+3]:f'Libellé Etendu Liste_{i}'}, inplace = True)
    df_data.rename(columns = {df_data.columns[start+4]:f'Nom Tête de Liste_{i}'}, inplace = True)
    df_data.rename(columns = {df_data.columns[start+5]:f'Voix_{i}'}, inplace = True)
    df_data.rename(columns = {df_data.columns[start+6]:f'% Voix/Ins_{i}'}, inplace = True)
    df_data.rename(columns = {df_data.columns[start+7]:f'% Voix/Exp_{i}'}, inplace = True)
    df_data.rename(columns = {df_data.columns[start+8]:f'Sièges_{i}'}, inplace = True)

# Checking first that the column names are the same:
data_ = np.sort(df_data['Libellé de la région'])
map_ = np.sort(df_map['nom'])
df_comp = pd.DataFrame({'Data': data_, 'Map': map_})
df_comp['Matching'] = (df_comp['Data'] == df_comp['Map'])
df_comp  

,Data,Map,Matching
0,Auvergne-Rhône-Alpes,Auvergne-Rhône-Alpes,True
1,Bourgogne-Franche-Comté,Bourgogne-Franche-Comté,True
2,Bretagne,Bretagne,True
3,Centre-Val de Loire,Centre-Val de Loire,True
4,Corse,Corse,True
5,Grand Est,Grand Est,True
6,Guadeloupe,Guadeloupe,True
7,Guyane,Guyane,True
8,Hauts-de-France,Hauts-de-France,True
9,La Réunion,La Réunion,True


In [4]:
df_data.rename(columns = {'Libellé de la région':'ID'}, inplace = True)
df_map.rename(columns = {'nom':'ID'}, inplace = True)

df = pd.merge(df_map, df_data, on='ID')

df = df[(df['ID'] != 'La Réunion') & (df['ID'] != 'Martinique') & (df['ID'] != 'Guadeloupe') & 
        (df['ID'] != 'Guyane')].reset_index(drop=True)

df = df.dropna(axis=1, how='all')

display(df)

,ID,geometry,Code de la région,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,...,Sièges_10,N°Liste_11,Nuance Liste_11,Libellé Abrégé Liste_11,Libellé Etendu Liste_11,Nom Tête de Liste_11,Voix_11,% Voix/Ins_11,% Voix/Exp_11,Sièges_11
0,Corse,"MULTIPOLYGON (((8.53996 42.23689, 8.54030 42.2...",94,239808,102921,42.92,136887,57.08,1144,0.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Île-de-France,"POLYGON ((1.44624 49.04639, 1.44945 49.04765, ...",11,7240948,5008807,69.17,2232141,30.83,36807,0.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hauts-de-France,"MULTIPOLYGON (((1.37983 50.06518, 1.38000 50.0...",32,4226927,2838834,67.16,1388093,32.84,34895,0.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nouvelle-Aquitaine,"MULTIPOLYGON (((-1.79102 43.37292, -1.79048 43...",75,4352306,2787124,64.04,1565182,35.96,39346,0.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Normandie,"MULTIPOLYGON (((-1.94877 49.71649, -1.94836 49...",28,2384818,1598056,67.01,786762,32.99,21254,0.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Pays de la Loire,"MULTIPOLYGON (((-2.55966 47.37430, -2.55941 47...",52,2775345,1922512,69.27,852833,30.73,24669,0.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Centre-Val de Loire,"POLYGON ((0.05272 47.19656, 0.05321 47.19721, ...",24,1816435,1221658,67.26,594777,32.74,16327,0.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Grand Est,"POLYGON ((3.38364 48.47958, 3.38370 48.47963, ...",44,3829952,2695773,70.39,1134179,29.61,34389,0.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Provence-Alpes-Côte d'Azur,"MULTIPOLYGON (((4.23014 43.46047, 4.23025 43.4...",93,3580639,2373318,66.28,1207321,33.72,33522,0.94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Bretagne,"MULTIPOLYGON (((-4.79551 48.41438, -4.79551 48...",53,2493602,1601187,64.21,892415,35.79,24211,0.97,...,0.0,13.0,LDSV,DEBOUT LA BRETAGNE,DEBOUT LA BRETAGNE DEBOUT LA FRANCE !,CABAS David,11919.0,0.48,1.4,0.0


In [5]:
nb_columns = df.shape[1]
number1 = []; number2 = []; number3 = []
for i in range(len(df['ID'])):
    tmp = []
    for j in range(0, int((nb_columns-16)/9)-1):
        tmp.append(-df[f'Voix_{j}'].iloc[i])
    number1.append(tmp.index(np.sort(tmp)[0]))
    number2.append(tmp.index(np.sort(tmp)[1])) 
    number3.append(tmp.index(np.sort(tmp)[2]))    

Create a new frame with the top list

In [6]:
NCandi_1 = []; NCandi_2 = []; NCandi_3 = []
NListe_1 = []; NListe_2 = []; NListe_3 = []
vpListe_1 = []; vpListe_2 = []; vpListe_3 = []

for i in range(len(df)):
    NCandi_1.append(df[f'Nom Tête de Liste_{number1[i]}'].iloc[i])
    NCandi_2.append(df[f'Nom Tête de Liste_{number2[i]}'].iloc[i])
    NCandi_3.append(df[f'Nom Tête de Liste_{number3[i]}'].iloc[i])

    NListe_1.append(df[f'Nuance Liste_{number1[i]}'].iloc[i])
    NListe_2.append(df[f'Nuance Liste_{number2[i]}'].iloc[i])
    NListe_3.append(df[f'Nuance Liste_{number3[i]}'].iloc[i])    
    
    vpListe_1.append(df[f'% Voix/Exp_{number1[i]}'].iloc[i])
    vpListe_2.append(df[f'% Voix/Exp_{number2[i]}'].iloc[i])
    vpListe_3.append(df[f'% Voix/Exp_{number3[i]}'].iloc[i])    
    
    
df_final = pd.DataFrame({'ID': df['ID'], 'geometry': df['geometry'], 
                         'Nom Candidat 1': NCandi_1, 'Nuance Liste 1': NListe_1, '% Voix/Exp 1': vpListe_1,
                         'Nom Candidat 2': NCandi_2, 'Nuance Liste 2': NListe_2, '% Voix/Exp 2': vpListe_2,                        
                         'Nom Candidat 3': NCandi_3, 'Nuance Liste 3': NListe_3, '% Voix/Exp 3': vpListe_3,                        
                        })
df_final

,ID,geometry,Nom Candidat 1,Nuance Liste 1,% Voix/Exp 1,Nom Candidat 2,Nuance Liste 2,% Voix/Exp 2,Nom Candidat 3,Nuance Liste 3,% Voix/Exp 3
0,Corse,"MULTIPOLYGON (((8.53996 42.23689, 8.54030 42.2...",SIMEONI Gilles,LREG,29.19,MARCANGELI Laurent,LDVD,24.86,ANGELINI Jean-Christophe,LREG,13.22
1,Île-de-France,"POLYGON ((1.44624 49.04639, 1.44945 49.04765, ...",PÉCRESSE Valérie,LUD,36.18,BAYOU Julien,LECO,12.97,SAINT-MARTIN Laurent,LUC,11.79
2,Hauts-de-France,"MULTIPOLYGON (((1.37983 50.06518, 1.38000 50.0...",BERTRAND Xavier,LUD,41.42,CHENU Sébastien,LRN,24.37,DELLI Karima,LUGE,18.97
3,Nouvelle-Aquitaine,"MULTIPOLYGON (((-1.79102 43.37292, -1.79048 43...",ROUSSET Alain,LUG,28.83,DARRIEUSSECQ Geneviève,LUC,13.71,FLORIAN Nicolas,LLR,12.48
4,Normandie,"MULTIPOLYGON (((-1.94877 49.71649, -1.94836 49...",MORIN Hervé,LUCD,36.86,BAY Nicolas,LRN,19.86,BOULANGER Mélanie,LUGE,18.37
5,Pays de la Loire,"MULTIPOLYGON (((-2.55966 47.37430, -2.55941 47...",MORANÇAIS Christelle,LUCD,34.30,GAROT Guillaume,LUG,16.32,JUVIN Hervé,LRN,12.54
6,Centre-Val de Loire,"POLYGON ((0.05272 47.19656, 0.05321 47.19721, ...",BONNEAU François,LUG,24.81,NIKOLIC Aleksandar,LRN,22.24,FORISSIER Nicolas,LUCD,18.82
7,Grand Est,"POLYGON ((3.38364 48.47958, 3.38370 48.47963, ...",ROTTNER Jean,LUCD,31.15,JACOBELLI Laurent,LRN,21.12,ROMANI Eliane,LUGE,14.60
8,Provence-Alpes-Côte d'Azur,"MULTIPOLYGON (((4.23014 43.46047, 4.23025 43.4...",MARIANI Thierry,LRN,36.38,MUSELIER Renaud,LLR,31.91,FELIZIA Jean-Laurent,LUGE,16.89
9,Bretagne,"MULTIPOLYGON (((-4.79551 48.41438, -4.79551 48...",CHESNAIS-GIRARD Loïg,LUG,20.95,BURLOT Thierry,LUC,15.53,DESMARES-POIRRIER Claire,LUGE,14.84


### GeoJSON file

In [7]:
df_final = gpd.GeoDataFrame(df_final, geometry=df_final['geometry'])
df_final.to_csv('final.csv')
df_final.to_file('data.json', driver='GeoJSON')

with open('data.json') as f:
    df_geo = json.load(f) 

### Figure

In [8]:
latitude = 46.2276; longitude = 2.2137
time_label = datetime.now().strftime(f'%d%m%Y_%Hh%Mm%S')

mymap = folium.Map(location=[latitude, longitude], zoom_start=5, tiles=None)

folium.Choropleth(
    geo_data=df_geo,
    name='ID',
    data=df_final,
    columns=['ID', '% Voix/Exp 1'],
    key_on='feature.properties.ID',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Voix/Exp (%)',
).add_to(mymap)

# Interactions

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
over = folium.features.GeoJson(
    df_geo,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['ID', 'Nom Candidat 1', 'Nuance Liste 1', '% Voix/Exp 1', 
               'Nom Candidat 2', 'Nuance Liste 2', '% Voix/Exp 2', 
               'Nom Candidat 3', 'Nuance Liste 3', '% Voix/Exp 3' 
               ],
        aliases=['Region: ', 'Candidat: ', 'Liste: ', 'Voix %: ', 
                 'Candidat 2: ', 'Liste: ', 'Voix %: ',
                 'Candidat 3: ',  'Liste: ', 'Voix %: '],
    )
)
mymap.add_child(over)
mymap.keep_in_front(over)
folium.LayerControl().add_to(mymap)

mymap.save(f'html_files/election_{time_label}.html')

mymap